# **CRAB Quickstart**

Goal: run a single CRAB optimization using the refactored `src/` workflows starting from the default baseline pulses.
Quickstart: edit the Parameter Panel, run all cells, then review the Results Summary.

Sections:
- [Parameter Panel](#Parameter-Panel)
- [Run Experiment](#Run-Experiment)
- [Standard Plots](#Standard-Plots)
- [Penalty History](#Penalty-History)
- [Results Summary](#Results-Summary)
- [Troubleshooting](#Troubleshooting)


## Imports


In [ ]:
# Imports
from pathlib import Path
import json
import numpy as np

from src import (
    ExperimentConfig,
    override_from_dict,
    run_experiment,
    plot_cost_history,
    plot_pulses,
    plot_summary,
    plot_penalties_history,
    Result,
)
from src.optimizers.base import load_crab_problem



## Parameter Panel


In [ ]:
# @title Parameter Panel
BASELINE_ARRAYS = Path('data/baselines/_baseline_crab/arrays.npz')
with np.load(BASELINE_ARRAYS) as baseline_npz:
    arrays = {key: np.array(baseline_npz[key]) for key in baseline_npz.files}

t_us = arrays['t'].astype(float)
T_US = float(arrays['T'])
M = int(arrays['Nt'])
delta0 = arrays['Delta0'].astype(float)
delta_max_rad_per_us = float(np.abs(delta0).max())
delta_max_MHz = delta_max_rad_per_us / (2 * np.pi)
default_K = int(arrays['CRAB_BASIS_OMEGA'].shape[1])
default_omegas = (np.arange(1, default_K + 1) * np.pi / T_US).tolist()

BASELINE_DATA = {
    't_us': t_us,
    'dt_us': float(np.diff(t_us).mean()),
    'omega_base': arrays['Omega0'].astype(float),
    'delta_base': delta0,
    'rho0': arrays['rho0'],
    'target': arrays['target'],
    'delta_max_rad_per_us': delta_max_rad_per_us,
}

user_params = {
    'run_name': None,
    'output_dir': 'results/quickstart',
    'objective': 'terminal',  # terminal | path | ensemble
    'time_grid': {'t0_us': float(t_us[0]), 't1_us': float(t_us[-1]), 'M': M},
    'crab': {
        'K': default_K,
        'omegas_rad_per_us': default_omegas,
        'phases': [0.0] * default_K,
        'use_delta': True,
    },
    'weights': {'w_power': 0.0, 'w_neg': 0.0, 'neg_kappa': 10.0},
    'optimizer': {
        'method': 'adam',
        'max_iters': 200,
        'learning_rate': 0.05,
        'beta1': 0.9,
        'beta2': 0.999,
        'epsilon': 1e-8,
        'grad_clip': None,
    },
    'path': {'reference': 'adiabatic_ground_state'},
    'ensemble': {
        'beta_min': 0.9,
        'beta_max': 1.1,
        'num_beta': 5,
        'detuning_MHz_min': -delta_max_MHz,
        'detuning_MHz_max': delta_max_MHz,
        'num_detuning': 5,
    },
}

overrides = {
    'baseline': {'name': 'default'},
    'run_name': user_params['run_name'],
    'artifacts_root': str(Path(user_params['output_dir']).resolve()),
    'optimizer_options': {
        'method': user_params['optimizer']['method'],
        'max_iters': int(user_params['optimizer']['max_iters']),
        'learning_rate': float(user_params['optimizer']['learning_rate']),
        'beta1': float(user_params['optimizer']['beta1']),
        'beta2': float(user_params['optimizer']['beta2']),
        'epsilon': float(user_params['optimizer']['epsilon']),
        'grad_clip': user_params['optimizer']['grad_clip'],
        'optimize_delta': bool(user_params['crab']['use_delta']),
        'K': int(user_params['crab']['K']),
        'omegas_rad_per_us': user_params['crab']['omegas_rad_per_us'],
        'phases': user_params['crab']['phases'],
    },
    'penalties': {
        'power_weight': float(user_params['weights']['w_power']),
        'neg_weight': float(user_params['weights']['w_neg']),
        'neg_kappa': float(user_params['weights']['neg_kappa']),
    },
    'metadata': {
        'objective': user_params['objective'],
        'path_params': user_params['path'],
        'ensemble_params': user_params['ensemble'],
    },
}

cfg = override_from_dict(ExperimentConfig(), overrides)
cfg.metadata.setdefault('baseline_arrays', str(BASELINE_ARRAYS))
cfg.metadata['delta_max_rad_per_us'] = delta_max_rad_per_us
print(json.dumps(cfg.to_dict(), indent=2))
cfg



## Run Experiment


In [ ]:
# @title Run Experiment {display-mode: "code"}
result: Result = run_experiment(cfg, method=user_params['optimizer']['method'])
result



## Standard Plots


In [ ]:
# @title Standard Plots
_ = plot_cost_history(result)
_ = plot_pulses(result)
_ = plot_summary(result)



## Penalty History


In [ ]:
# @title Penalty History
_ = plot_penalties_history(result)



## Results Summary


In [ ]:
    # @title Results Summary
    from pprint import pprint

    metrics_table = {
        'terminal_infidelity': result.final_metrics.get('terminal'),
        'path_infidelity': result.final_metrics.get('path'),
        'ensemble_infidelity': result.final_metrics.get('ensemble'),
        'power_penalty': result.final_metrics.get('power_penalty'),
        'neg_penalty': result.final_metrics.get('neg_penalty'),
        'total_cost': result.final_metrics.get('total'),
        'runtime_seconds': result.final_metrics.get('runtime_s'),
    }
    print('Final metrics:')
    for key, value in metrics_table.items():
        if value is not None:
            print(f"  {key:>20}: {value:.6g}")
        else:
            print(f"  {key:>20}: n/a")
    print('
Artifacts saved in', result.artifacts_dir)
    for artifact in sorted(Path(result.artifacts_dir).iterdir()):
        print(' •', artifact.name)



## Troubleshooting

- Learning rate too aggressive? Decrease `learning_rate` (Adam/const) or `alpha0` (line-search).
- Cost plateaus early? Raise `max_iters` or relax penalty weights so the optimizer can explore.
- Armijo fails repeatedly? Tune `ls_beta` (0.3–0.7) and `ls_sigma` (1e-4–1e-2).
- Negativity penalty dominates? Reduce `w_neg` or increase `neg_kappa`.
- Basis too restrictive? Increase `K` or supply custom `omegas_rad_per_us`/`phases`.
- Detuning updates unstable? Set `use_delta` to `False` to keep the baseline detuning.
- Ensemble sweeps slow? Trim `num_beta` / `num_detuning` or enable `grad_clip`.
- Large artifacts folder? Change `output_dir` or reuse an existing `run_name` with `cfg` edits.
